In [2]:
import tensorflow as tf
import numpy as np 
import os

from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [3]:
from fast_scnn import fastSCNN, fastSCNN_edition2
from fast_scnn import *
from network import network_big

In [4]:
model_path = "./trained_models/20200825/427_0.1.h5"

In [5]:
input_shape=(256, 192, 3)

In [6]:
model = fastSCNN(input_shape, train=False)
model.load_weights(model_path, by_name=True)
print(model.output)
# model = network_big(input_shape, train=False)

Tensor("conv2d_35/Identity:0", shape=(None, 256, 192, 1), dtype=float32)


In [7]:
# model.summary()

In [8]:
withGraph_model = model_path.replace(".h5", "_withGraph.h5")
model.save(withGraph_model)

**실행 시간 체크**

In [9]:
x = np.random.random(size=(1,) + input_shape)

In [10]:
%%timeit # 기존 모델의 실행시간
pred = model.predict(x)

58 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### To Convert TFJS model

In [11]:
tf_saved = model_path.replace(".h5", "_InTF_savedModel")

In [12]:
# save tf-saved model
tf.saved_model.save(model, tf_saved)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./trained_models/20200825/427_0.1_InTF_savedModel/assets


In [13]:
import tensorflowjs
tensorflowjs.converters.convert_tf_saved_model(tf_saved, model_path.replace(".h5", "_tfjs_models"))

Writing weight file ./trained_models/20200825/427_0.1_tfjs_models/model.json...


### To Conver TFLITE model

In [14]:
tflite_file_name = model_path.replace(".h5", "_tflite.tflite")
tflite_file_name_4channel = model_path.replace(".h5", "_4channel_tflite.tflite")

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

with open(tflite_file_name, "wb") as f:
    f.write(tflite_model)
    
############ 4channel ##########################
model_4channel = fastSCNN(input_shape, train=True, android=True)
model_4channel.load_weights(model_path, by_name=False)
model_4channel = tf.keras.models.Model(model_4channel.input, model_4channel.output[0])

converter = tf.lite.TFLiteConverter.from_keras_model(model_4channel)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

with open(tflite_file_name_4channel, "wb") as f:
    f.write(tflite_model)

### To Conver mlmodel


In [16]:
import coremltools

In [17]:
coreml_model_file = model_path.replace(".h5", "_coreml.mlmodel")

In [18]:
ml_model = coremltools.converters.tensorflow.convert(withGraph_model,
                                                input_names='input_1', 
                                                image_input_names='input_1', 
                                                red_bias = -(123.68) / 58.393,
                                                green_bias = -(116.779) / 57.12, 
                                                blue_bias = -(103.939) / 57.375,
                                                image_scale = 1.0 / 57.63,
                                                custom_conversion_functions={"BilinearInterpolation":BilinearInterpolation}
                                               )
ml_model.save(coreml_model_file)

0 assert nodes deleted
297 nodes deleted
150 nodes deleted
0 nodes deleted
[Op Fusion] fuse_bias_add() deleted 88 nodes.
1 identity nodes deleted
36 disconnected nodes deleted
[Op Fusion] Fused 294 nodes into BatchNorms.
[SSAConverter] Converting function main ...
[SSAConverter] [1/198] Converting op type: 'Placeholder', name: 'input_1', output_shape: (-1, 256, 192, 3).
[SSAConverter] [2/198] Converting op type: 'Const', name: 'model/depthwise_conv2d/depthwise/ReadVariableOp', output_shape: (3, 3, 32, 1).
[SSAConverter] [3/198] Converting op type: 'Const', name: 'model/depthwise_conv2d_1/depthwise/ReadVariableOp', output_shape: (3, 3, 48, 1).
[SSAConverter] [4/198] Converting op type: 'Const', name: 'model/depthwise_conv2d_2/depthwise/ReadVariableOp', output_shape: (3, 3, 384, 1).
[SSAConverter] [5/198] Converting op type: 'Const', name: 'model/depthwise_conv2d_3/depthwise/ReadVariableOp', output_shape: (3, 3, 384, 1).
[SSAConverter] [6/198] Converting op type: 'Const', name: 'model/de

[SSAConverter] [115/198] Converting op type: 'BatchNorm', name: 'model/batch_normalization_28/FusedBatchNormV3/Add_batch_norm', output_shape: (-1, 128, 8, 6).
[SSAConverter] [116/198] Converting op type: 'AddV2', name: 'model/tf_op_layer_add_4/add_4', output_shape: (-1, 128, 8, 6).
[SSAConverter] [117/198] Converting op type: 'Conv2D', name: 'model/conv2d_19/Conv2D', output_shape: (-1, 768, 8, 6).
[SSAConverter] [118/198] Converting op type: 'BatchNorm', name: 'model/batch_normalization_29/FusedBatchNormV3/Add_batch_norm', output_shape: (-1, 768, 8, 6).
[SSAConverter] [119/198] Converting op type: 'Relu', name: 'model/activation_21/Relu', output_shape: (-1, 768, 8, 6).
[SSAConverter] [120/198] Converting op type: 'DepthwiseConv2dNative', name: 'model/depthwise_conv2d_10/depthwise', output_shape: (-1, 768, 8, 6).
[SSAConverter] [121/198] Converting op type: 'BatchNorm', name: 'model/batch_normalization_30/FusedBatchNormV3/Add_batch_norm', output_shape: (-1, 768, 8, 6).
[SSAConverter] [1

/Users/MAC/.pyenv/versions/3.6.9/envs/tf20/lib/python3.6/site-packages/coremltools/models/model.py:112: RuntimeWarning: You will not be able to run predict() on this Core ML model. Underlying exception message was: Error compiling model: "Error reading protobuf spec. validator error: The .mlmodel supplied is of version 4, intended for a newer version of Xcode. This version of Xcode supports model version 3 or earlier.".
  RuntimeWarning)
